In [1]:
from __future__ import print_function

from collections import defaultdict
try:
    import cPickle as pickle
except ImportError:
    import pickle
from PIL import Image

from six.moves import range

import keras.backend as K
from keras.datasets import mnist
from keras import layers
from keras.layers import Input, Dense, Reshape, Flatten, Embedding, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils.generic_utils import Progbar
import numpy as np
import time, os
np.random.seed(1337)

Using TensorFlow backend.
/home/narendra/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/narendra/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/narendra/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/narendra/.local/lib/python3.6/site-packages/tensorflo

In [2]:
import pandas as pd
 #(x_train, y_train), (x_test, y_test)
x_train1=pd.read_csv("x_train_mix.csv")
y_train1=pd.read_csv("y_train_mix.csv")
x_test1=pd.read_csv("x_test_mix.csv")
y_test1=pd.read_csv("y_test_mix.csv")
  

In [3]:
x_train = np.genfromtxt("x_train_mix.csv", delimiter=",", skip_header=1)
y_train = np.genfromtxt("y_train_mix.csv", delimiter=",", skip_header=1)
x_test= np.genfromtxt("x_test_mix.csv", delimiter=",", skip_header=1)
y_test = np.genfromtxt("y_test_mix.csv", delimiter=",", skip_header=1)

In [4]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(39999, 121)
(39999,)
(9998, 121)
(9998,)


In [5]:
#import numpy as np
#data3 = np.arange(122304).reshape(52, 2352 )
x_train = x_train.reshape(39999,1, 11, 11)
#y_train = x_train.reshape(9999,1, 11, 11)
x_test = x_test.reshape(9998,1, 11, 11)
#y_test = x_test.reshape(4999,1, 11, 11)
print(x_train.shape)

(39999, 1, 11, 11)


In [6]:
np.random.seed(1331)

K.set_image_dim_ordering('th')
def modified_binary_crossentropy(target, output):
    #output = K.clip(output, _EPSILON, 1.0 - _EPSILON)
    #return -(target * output + (1.0 - target) * (1.0 - output))
    return K.mean(target*output)

In [7]:
K.set_image_data_format('channels_first')

num_classes = 10

In [8]:
def build_generator(latent_size):
    # we will map a pair of (z, L), where z is a latent vector and L is a
    # label drawn from P_c, to image space (..., 1, 28, 28)
    cnn = Sequential()

    cnn.add(Dense(1024, input_dim=latent_size, activation='relu'))
    cnn.add(Dense(128 * 11* 11, activation='relu'))
    cnn.add(Reshape((128, 11, 11)))

    #upsample to (..., 14, 14)
    #cnn.add(UpSampling2D(size=(2, 2)))
    cnn.add(Conv2D(256, 5, padding='same',
                   activation='relu',
                   kernel_initializer='glorot_normal'))

    #upsample to (..., 28, 28)
    #cnn.add(UpSampling2D(size=(1,1)))
    cnn.add(Conv2D(128, 5, padding='same',
                   activation='relu',
                   kernel_initializer='glorot_normal'))

    # take a channel axis reduction
    cnn.add(Conv2D(1, 2, padding='same',
                   activation='tanh',
                   kernel_initializer='glorot_normal'))

    # this is the z space commonly refered to in GAN papers
    latent = Input(shape=(latent_size, ))

    # this will be our label
    image_class = Input(shape=(1,), dtype='int32')

    cls = Flatten()(Embedding(num_classes, latent_size,
                              embeddings_initializer='glorot_normal')(image_class))

    # hadamard product between z-space and a class conditional embedding
    h = layers.multiply([latent, cls])
    #print(h)
    fake_image = cnn(h)

    return Model([latent, image_class], fake_image)

In [9]:

g=build_generator(100)
g.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 100)       1000        input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 100)          0           embedding_1[0][0]                
_____________________________________

In [10]:

def build_discriminator():
    # build a relatively standard conv net, with LeakyReLUs as suggested in
    # the reference paper
    cnn = Sequential()

    cnn.add(Conv2D(32, 3, padding='same', strides=2,
                   input_shape=(1, 11, 11)))
    cnn.add(LeakyReLU())
    cnn.add(Dropout(0.3))

    cnn.add(Conv2D(64, 3, padding='same', strides=1))
    cnn.add(LeakyReLU())
    cnn.add(Dropout(0.3))

    cnn.add(Conv2D(128, 3, padding='same', strides=2))
    cnn.add(LeakyReLU())
    cnn.add(Dropout(0.3))

    cnn.add(Conv2D(256, 3, padding='same', strides=1))
    cnn.add(LeakyReLU())
    cnn.add(Dropout(0.3))

    cnn.add(Flatten())

    image = Input(shape=(1, 11, 11))

    features = cnn(image)

    # first output (name=generation) is whether or not the discriminator
    # thinks the image that is being shown is fake, and the second output
    # (name=auxiliary) is the class that the discriminator thinks the image
    # belongs to.
    fake = Dense(1, activation='sigmoid', name='generation')(features) # fake oder nicht fake
    aux = Dense(num_classes, activation='softmax', name='auxiliary')(features) #welche klasse ist es

    return Model(image, [fake, aux])


In [11]:
d =build_discriminator()
d.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1, 11, 11)    0                                            
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 2304)         387840      input_3[0][0]                    
__________________________________________________________________________________________________
generation (Dense)              (None, 1)            2305        sequential_2[1][0]               
__________________________________________________________________________________________________
auxiliary (Dense)               (None, 10)           23050       sequential_2[1][0]          

In [18]:
if __name__ == '__main__':
    start_time_string = time.strftime("%Y_%m_%d_%H_%M_%S", time.gmtime())
    
    try:
        os.mkdir('history/' + start_time_string)
        os.mkdir('images/' + start_time_string)
        os.mkdir('acgan/' + start_time_string)
    except OSError:
        pass
    #os.mkdir('history/' + start_time_string)
    
    
    # batch and latent size taken from the paper
    epochs = 100
    batch_size = 100
    latent_size = 100

    # Adam parameters suggested in https://arxiv.org/abs/1511.06434
    adam_lr = 0.00005
    adam_beta_1 = 0.5

    # build the discriminator
    discriminator = build_discriminator()
    discriminator.compile(
        optimizer=Adam(lr=adam_lr, beta_1=adam_beta_1),
        loss=['binary_crossentropy', 'sparse_categorical_crossentropy']
    )

    # build the generator
    generator = build_generator(latent_size)
    generator.compile(optimizer=Adam(lr=adam_lr, beta_1=adam_beta_1),
                      loss='binary_crossentropy')

    latent = Input(shape=(latent_size, ))
    image_class = Input(shape=(1,), dtype='int32')

    # get a fake image
    fake = generator([latent, image_class])

    # we only want to be able to train generation for the combined model
    discriminator.trainable = False
    fake, aux = discriminator(fake)
    combined = Model([latent, image_class], [fake, aux])

    combined.compile(
        optimizer=Adam(lr=adam_lr, beta_1=adam_beta_1),
        loss=['binary_crossentropy', 'sparse_categorical_crossentropy']
    )

    # get our mnist data, and force it to be of shape (..., 1, 28, 28) with
    # range [-1, 1]
    
    #(x_train, y_train), (x_test, y_test) = mnist.load_data()
    
    x_train = (x_train.astype(np.float32) - 127.5) / 127.5
    #x_train = np.expand_dims(x_train, axis=1)
    print(x_train.shape)
    x_test = (x_test.astype(np.float32) - 127.5) / 127.5
    #x_test = np.expand_dims(x_test, axis=1)
    print(x_test.shape)
    num_train, num_test = x_train.shape[0], x_test.shape[0]

    train_history = defaultdict(list)
    test_history = defaultdict(list)

    for epoch in range(1, epochs + 1):
        print('Epoch {}/{}'.format(epoch, epochs))

        num_batches = int(x_train.shape[0] / batch_size)
        progress_bar = Progbar(target=num_batches)

        epoch_gen_loss = []
        epoch_disc_loss = []

        for index in range(num_batches):
            # generate a new batch of noise
            noise = np.random.uniform(-1, 1, (batch_size, latent_size))

            # get a batch of real images
            image_batch = x_train[index * batch_size:(index + 1) * batch_size]
            label_batch = y_train[index * batch_size:(index + 1) * batch_size]
            #print(image_batch.shape)
            #print(label_batch.shape)
            # sample some labels from p_c
            sampled_labels = np.random.randint(0, num_classes, batch_size)

            # generate a batch of fake images, using the generated labels as a
            # conditioner. We reshape the sampled labels to be
            # (batch_size, 1) so that we can feed them into the embedding
            # layer as a length one sequence
            generated_images = generator.predict(
                [noise, sampled_labels.reshape((-1, 1))], verbose=0)
            #print(image_batch.shape)
            #print(generated_images.shape)
            x = np.concatenate((image_batch, generated_images))
            y = np.array([1] * batch_size + [0] * batch_size)
            print(label_batch.shape)
            print(sampled_labels.shape)
            aux_y = np.concatenate((label_batch, sampled_labels), axis=0)

            # see if the discriminator can figure itself out...
            epoch_disc_loss.append(discriminator.train_on_batch(x, [y, aux_y]))

            # make new noise. we generate 2 * batch size here such that we have
            # the generator optimize over an identical number of images as the
            # discriminator
            noise = np.random.uniform(-1, 1, (2 * batch_size, latent_size))
            sampled_labels = np.random.randint(0, num_classes, 2 * batch_size)

            # we want to train the generator to trick the discriminator
            # For the generator, we want all the {fake, not-fake} labels to say
            # not-fake
            trick = np.ones(2 * batch_size)

            epoch_gen_loss.append(combined.train_on_batch(
                [noise, sampled_labels.reshape((-1, 1))],
                [trick, sampled_labels]))

            progress_bar.update(index + 1)

        print('Testing for epoch {}:'.format(epoch))

        # evaluate the testing loss here

        # generate a new batch of noise
        noise = np.random.uniform(-1, 1, (num_test, latent_size))

        # sample some labels from p_c and generate images from them
        sampled_labels = np.random.randint(0, num_classes, num_test)
        generated_images = generator.predict(
            [noise, sampled_labels.reshape((-1, 1))], verbose=False)

        x = np.concatenate((x_test, generated_images))
        y = np.array([1] * num_test + [0] * num_test)
        aux_y = np.concatenate((y_test, sampled_labels), axis=0)

        # see if the discriminator can figure itself out...
        discriminator_test_loss = discriminator.evaluate(
            x, [y, aux_y], verbose=False)

        discriminator_train_loss = np.mean(np.array(epoch_disc_loss), axis=0)

        # make new noise
        noise = np.random.uniform(-1, 1, (2 * num_test, latent_size))
        sampled_labels = np.random.randint(0, num_classes, 2 * num_test)

        trick = np.ones(2 * num_test)

        generator_test_loss = combined.evaluate(
            [noise, sampled_labels.reshape((-1, 1))],
            [trick, sampled_labels], verbose=False)

        generator_train_loss = np.mean(np.array(epoch_gen_loss), axis=0)

        # generate an epoch report on performance
        train_history['generator'].append(generator_train_loss)
        train_history['discriminator'].append(discriminator_train_loss)

        test_history['generator'].append(generator_test_loss)
        test_history['discriminator'].append(discriminator_test_loss)

        print('{0:<22s} | {1:4s} | {2:15s} | {3:5s}'.format(
            'component', *discriminator.metrics_names))
        print('-' * 65)

        ROW_FMT = '{0:<22s} | {1:<4.2f} | {2:<15.2f} | {3:<5.2f}'
        print(ROW_FMT.format('generator (train)',
                             *train_history['generator'][-1]))
        print(ROW_FMT.format('generator (test)',
                             *test_history['generator'][-1]))
        print(ROW_FMT.format('discriminator (train)',
                             *train_history['discriminator'][-1]))
        print(ROW_FMT.format('discriminator (test)',
                             *test_history['discriminator'][-1]))

        # save weights every epoch
        #generator.save_weights(
           # 'acgan/'+ start_time_string +'/params_generator_epoch_{0:03d}.hdf5'.format(epoch), True)
        #discriminator.save_weights(
            #'acgan/'+ start_time_string +'/params_discriminator_epoch_{0:03d}.hdf5'.format(epoch), True)

        # generate some digits to display
        noise = np.random.uniform(-1, 1, (100, latent_size))

        sampled_labels = np.array([
            [i] * num_classes for i in range(num_classes)
        ]).reshape(-1, 1)

        # get a batch to display
        generated_images = generator.predict(
            [noise, sampled_labels], verbose=0)

        print("**************")
        count = 0
        # arrange them into a grid
        print(generated_images.shape[0])
        print(generated_images)
        for i in range(generated_images.shape[0]):
            timg = (generated_images[i].reshape(11,11) * 127.5 + 127.5).astype(np.uint8)
            Image.fromarray(timg).convert("L").save(
                'images/'+ start_time_string +'/plot_epoch_{0:03d}_generated_{1}.png'.format(epoch,i+1))

    #pickle.dump({'train': train_history, 'test': test_history},
                #open('history/'+ start_time_string +'/acgan-history.pkl', 'wb'))

(39999, 1, 11, 11)
(9998, 1, 11, 11)
Epoch 1/100
(100,)
(100,)
  1/399 [..............................] - ETA: 55:53(100,)
(100,)
  2/399 [..............................] - ETA: 37:34(100,)
(100,)
  3/399 [..............................] - ETA: 31:02(100,)
(100,)
  4/399 [..............................] - ETA: 27:37(100,)
(100,)
  5/399 [..............................] - ETA: 25:35(100,)
(100,)
  6/399 [..............................] - ETA: 24:13(100,)
(100,)
  7/399 [..............................] - ETA: 23:08(100,)
(100,)
  8/399 [..............................] - ETA: 22:29(100,)
(100,)
  9/399 [..............................] - ETA: 21:59(100,)
(100,)
 10/399 [..............................] - ETA: 21:35(100,)
(100,)
 11/399 [..............................] - ETA: 21:20(100,)
(100,)
 12/399 [..............................] - ETA: 21:00(100,)
(100,)
 13/399 [..............................] - ETA: 20:43(100,)
(100,)
 14/399 [>.............................] - ETA: 20:26(100,)
(100,)

245/399 [=================>............] - ETA: 6:49(100,)
(100,)
246/399 [=================>............] - ETA: 6:46(100,)
(100,)
247/399 [=================>............] - ETA: 6:43(100,)
(100,)
248/399 [=================>............] - ETA: 6:41(100,)
(100,)
249/399 [=================>............] - ETA: 6:39(100,)
(100,)
250/399 [=================>............] - ETA: 6:37(100,)
(100,)
251/399 [=================>............] - ETA: 6:35(100,)
(100,)
252/399 [=================>............] - ETA: 6:33(100,)
(100,)
253/399 [==================>...........] - ETA: 6:30(100,)
(100,)
254/399 [==================>...........] - ETA: 6:28(100,)
(100,)
255/399 [==================>...........] - ETA: 6:25(100,)
(100,)
256/399 [==================>...........] - ETA: 6:22(100,)
(100,)
257/399 [==================>...........] - ETA: 6:20(100,)
(100,)
258/399 [==================>...........] - ETA: 6:17(100,)
(100,)
259/399 [==================>...........] - ETA: 6:15(100,)
(100,)
260/399 [=

 40/399 [==>...........................] - ETA: 16:52(100,)
(100,)
 41/399 [==>...........................] - ETA: 16:49(100,)
(100,)
 42/399 [==>...........................] - ETA: 16:45(100,)
(100,)
 43/399 [==>...........................] - ETA: 16:42(100,)
(100,)
 44/399 [==>...........................] - ETA: 16:40(100,)
(100,)
 45/399 [==>...........................] - ETA: 16:38(100,)
(100,)
 46/399 [==>...........................] - ETA: 16:37(100,)
(100,)
 47/399 [==>...........................] - ETA: 16:34(100,)
(100,)
 48/399 [==>...........................] - ETA: 16:32(100,)
(100,)
 49/399 [==>...........................] - ETA: 16:28(100,)
(100,)
 50/399 [==>...........................] - ETA: 16:25(100,)
(100,)
 51/399 [==>...........................] - ETA: 16:22(100,)
(100,)
 52/399 [==>...........................] - ETA: 16:19(100,)
(100,)
 53/399 [==>...........................] - ETA: 16:17(100,)
(100,)
 54/399 [===>..........................] - ETA: 16:14(100,)
(1

286/399 [====================>.........] - ETA: 4:46(100,)
(100,)
287/399 [====================>.........] - ETA: 4:44(100,)
(100,)
288/399 [====================>.........] - ETA: 4:41(100,)
(100,)
289/399 [====================>.........] - ETA: 4:39(100,)
(100,)
290/399 [====================>.........] - ETA: 4:36(100,)
(100,)
291/399 [====================>.........] - ETA: 4:33(100,)
(100,)
292/399 [====================>.........] - ETA: 4:31(100,)
(100,)
293/399 [=====================>........] - ETA: 4:28(100,)
(100,)
294/399 [=====================>........] - ETA: 4:25(100,)
(100,)
295/399 [=====================>........] - ETA: 4:23(100,)
(100,)
296/399 [=====================>........] - ETA: 4:20(100,)
(100,)
297/399 [=====================>........] - ETA: 4:17(100,)
(100,)
298/399 [=====================>........] - ETA: 4:15(100,)
(100,)
299/399 [=====================>........] - ETA: 4:12(100,)
(100,)
300/399 [=====================>........] - ETA: 4:09(100,)
(100,)
301/399 [=

(100,)
(100,)
  1/399 [..............................] - ETA: 18:51(100,)
(100,)
  2/399 [..............................] - ETA: 18:27(100,)
(100,)
  3/399 [..............................] - ETA: 18:28(100,)
(100,)
  4/399 [..............................] - ETA: 18:31(100,)
(100,)
  5/399 [..............................] - ETA: 18:43(100,)
(100,)
  6/399 [..............................] - ETA: 18:48(100,)
(100,)
  7/399 [..............................] - ETA: 18:10(100,)
(100,)
  8/399 [..............................] - ETA: 17:40(100,)
(100,)
  9/399 [..............................] - ETA: 17:44(100,)
(100,)
 10/399 [..............................] - ETA: 17:51(100,)
(100,)
 11/399 [..............................] - ETA: 17:48(100,)
(100,)
 12/399 [..............................] - ETA: 17:47(100,)
(100,)
 13/399 [..............................] - ETA: 17:48(100,)
(100,)
 14/399 [>.............................] - ETA: 17:43(100,)
(100,)
 15/399 [>.............................] - ETA: 

KeyboardInterrupt: 

In [2]:
os.mkdir('images/' )
os.mkdir('acgan/' )
os.mkdir('history/')